# 目录

+ 1 遍历单个域名
    + 1.1 提取维基百科网站任一页面的所有词条链接
    + 1.2 包含了函数的程序
+ 2 采集整个网站
    + 2.1 链接去重
    + 2.2 收集整个网站数据
    + 2.3 通过互联网采集
        + 2.3.1 从 http://oreilly.com 开始，随机地从一个外链跳到另一个外链
        + 2.3.2 获取一个网站的所有内链和外链

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

# 1 遍历单个域名

## 1.1 提取维基百科网站任一页面的所有词条链接

In [2]:
html = urlopen("http://en.wikipedia.org/wiki/Kevin_Bacon") # 以 Kevin Bacon 这一词条所在页面为例
bsObj = BeautifulSoup(html, "lxml")

for link in bsObj.findAll("a"):
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
http://baconbros.com/
#cite_note-1
#cite_note-actor-2
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Awards
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
#cite_note-3
/wiki/Hollywood_Walk_of_Fame
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
#cite_note-walk-4
#Early_life_and_education
#Acting_career
#Early_work
#1980s
#1990s
#2000s
#2010s
#Advertising_work
#Personal_life
#Six_Degrees_of_Kevin_Bacon
#Music
#Awards_and_nominations
#Refere

维基百科的每个页面都充满了侧边栏、页眉、页脚链接，以及连接到分类页面、对话页面和其他不包含词条的页面链接，需要将这些页面过滤掉，以获取词条页面的链接：  
`/wiki/Category:Articles_with_unsourced_statements_from_April_2014`  
`/wiki/Talk:Kevin_Bacon`  

观察可以发现，那些指向词条页面（不是指向其他内容页面）的链接，会发现它们都有三个共同点：  
+ 它们都在 id 是 bodyContent 的 div 标签里
+ URL 链接不包含分号
+ URL 链接都以 /wiki/ 开头

In [3]:
for link in bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")): # 设置规则过滤掉不包含词条的页面链接
    if 'href' in link.attrs:
        print(link.attrs['href'])

/wiki/Kevin_Bacon_(disambiguation)
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Awards
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
/wiki/Hollywood_Walk_of_Fame
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
/wiki/Philadelphia
/wiki/Edmund_Bacon_(architect)
/wiki/Pennsylvania_Governor%27s_School_for_the_Arts
/wiki/Bucknell_University
/wiki/Glory_Van_Scott
/wiki/Kevin_Bacon_filmography
/wiki/Circle_in_the_Square
/wiki/Nancy_Mills
/wiki/Cosmopolitan_(magazine)
/wiki/Fraternities_and_sororities
/wiki/Animal_House
/wiki/Search_for_Tomorrow
/wiki/Guiding_Light
/wiki/

## 1.2 包含了函数的程序

+ 一个函数 `getLinks` ，可以用维基百科词条 `/wiki/< 词条名称 >` 形式的 `URL` 链接作为参数，然后以同样的形式返回一个列表，里面包含所有的词条 `URL` 链接。
+ 一个主函数， 以某个起始词条为参数调用 `getLinks` ，再从返回的 `URL` 列表里随机选择一个词条链接，再调用 `getLinks` ，直到我们主动停止，或者在新的页面上没有词条链接了，程序才停止运行。

In [4]:
random.seed(datetime.datetime.now()) # 根据系统时间来生成随机数种子，确保每次程序运行的时候，维基百科词条的选择都是一个全新的随机路径

In [5]:
# getLinks 函数，可以用维基百科词条 /wiki/< 词条名称 > 形式的 URL 链接作为参数，然后以同样的形式返回一个列表，里面包含所有的词条 URL 链接
def getLinks(articleUrl):
    html = urlopen("http://en.wikipedia.org" + articleUrl)
    bsObj = BeautifulSoup(html, "lxml")
    return bsObj.find("div", {"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$"))

In [6]:
links = getLinks("/wiki/Kevin_Bacon")

In [9]:
# 主函数，以某个起始词条为参数调用 getLinks，再从返回的 URL 列表里随机选择一个词条链接
# 再调用 getLinks，直到我们主动停止，或者在新的页面上没有词条链接了，程序才停止运行
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newArticle)
    links = getLinks(newArticle)

/wiki/Tel_Aviv
/wiki/Tel_Aviv_(disambiguation)
/wiki/Tel_Aviv
/wiki/Hebrew_language
/wiki/Hebrew_(disambiguation)
/wiki/File:Wiktionary-logo-v2.svg
/wiki/File:Commons-logo.svg
/wiki/Wikipedia:Criteria_for_speedy_deletion#F8
/wiki/Help:Edit_summary
/wiki/Wikipedia:Information_pages


KeyboardInterrupt: 

# 2 采集整个网站

## 2.1 链接去重
为了避免一个页面被采集两次，在代码运行时，把已发现的所有链接都放到一起，并保存在方便查询的列表里（下文示例指 `Python` 的集合 `set` 类型）。只
有“新”链接才会被采集，之后再从页面中搜索其他链接

In [8]:
pages = set() # 用集合 set 进行去重

def getLinks(pageUrl):
    
    global pages
    
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "lxml")
    
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # 我们遇到了新页面
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks("")

/wiki/Wikipedia
/wiki/Wikipedia:Protection_policy#semi
/wiki/Wikipedia:Requests_for_page_protection
/wiki/Wikipedia:Requests_for_permissions
/wiki/Wikipedia:Requesting_copyright_permission
/wiki/Wikipedia:User_access_levels
/wiki/Wikipedia:Requests_for_adminship
/wiki/Wikipedia:Requested_articles
/wiki/Wikipedia:Recent_additions
/wiki/Wikipedia:Shortcut


KeyboardInterrupt: 

一开始，用 `getLinks` 处理一个空 `URL` ，其实是维基百科的主页，因为在函数里空 `URL` 就是 `http://en.wikipedia.org` 。然后，遍历首页上每个链接，并检查是否已经在全局变量集合 `pages` 里面了（已经采集的页面集合）。如果不在，就打印到屏幕上，并把链接加入 `pages` 集合，再用 `getLinks` 递归地处理这个链接。

## 2.2 收集整个网站数据
创建一个爬虫来收集页面标题、正文的第一个段落，以及编辑页面的链接（如果有的话）

先观察网站上的一些页面，然后拟定一个采集模式。通过观察几个维基百科页面，包括词条和非词条页面，比如隐私策略之类的页面，就会得出下面的规则：
+ 所有的标题（ 所有页面上，不论是词条页面、编辑历史页面还是其他页面）都是在 `h1` → `span` 标签里，而且页面上只有一个 `h1` 标签。
+ 前面提到过， 所有的正文文字都在 `div#bodyContent` 标签里。但是，如果我们想更进一步获取第一段文字， 可能用 `div#mw-content-text` → `p` 更好（只选择第一段的标签）。这个规则对所有页面都适用， 除了文件页面（例如， `https://en.wikipedia.org/wiki/File:Orbit_of_274301_Wikipedia.svg` ），页面不包含内容文字（ `content text` ）的部分内容。
+ 编辑链接只出现在词条页面上。 如果有编辑链接，都位于 `li#ca-edit` 标签的 `li#caedit` → `span` → `a` 里面。

因为我们不可能确保每一页上都有所有类型的数据，所以每个打印语句都是按照数据在页面上出现的可能性从高到低排列的：  
 + `<h1>` 标题标签会出现在每一页上（只要能识别，无论哪一页都有），所以我们首先试着获取它的数据。
 + 正文内容会出现在大多数页面上（ 除了文件页面），因此是第二个获取的数据。
 + “编辑”按钮只出现在标题和正文内容都已经获取的页面上，但不是所有这类页面上都有，所以我们最后打印这类数据。

In [16]:
pages = set()

def getLinks(pageUrl):
    
    global pages
    html = urlopen("http://en.wikipedia.org"+pageUrl)
    bsObj = BeautifulSoup(html, "lxml")
    
    #这种按照网站上信息出现的可能性高低进行排序的方法对许多网站都是可行的，偶而会丢失一点儿数据，只要保存详细的日志就不是什么问题了
    try:
        print(bsObj.h1.get_text()) # 标题
        print(bsObj.find(id="mw-content-text").findAll("p")[0]) # 第一段文字所在标签
        print(bsObj.find(id="ca-edit").find("span").find("a").attrs['href']) # 标记链接
    except AttributeError:
        print("页面缺少一些属性！不过不用担心！ ")

    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                # 我们遇到了新页面
                newPage = link.attrs['href']
                print("----------------\n"+newPage)
                pages.add(newPage)
                getLinks(newPage)

getLinks("")

Main Page
<p><i>Part of the <b><a href="/wiki/Wikipedia:Featured_topics/God_of_War_franchise" title="Wikipedia:Featured topics/God of War franchise">God of War franchise</a></b> series, one of Wikipedia's <a href="/wiki/Wikipedia:Featured_topics" title="Wikipedia:Featured topics">featured topics</a>.</i></p>
页面缺少一些属性！不过不用担心！ 
----------------
/wiki/Wikipedia
Wikipedia
<p><b>Wikipedia</b> (<span class="nowrap"><span class="noexcerpt"><a href="//upload.wikimedia.org/wikipedia/commons/0/01/GT_Wikipedia_BE.ogg" title="Listen"><img alt="Listen" data-file-height="11" data-file-width="11" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Speakerlink-new.svg/11px-Speakerlink-new.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Speakerlink-new.svg/17px-Speakerlink-new.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Speakerlink-new.svg/22px-Speakerlink-new.svg.png 2x" width="11"/></a><sup><span class="IPA" style="color:#00e;font:bold 80% s

KeyboardInterrupt: 

## 2.3 通过互联网采集

### 2.3.1 从 `http://oreilly.com` 开始，随机地从一个外链跳到另一个外链

![](https://raw.githubusercontent.com/blueliberty/Web-Scraping/master/Pictures/followExternalOnly.png)

In [27]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

# 获取页面所有内链的列表
def getInternalLinks(bsObj, includeUrl):
    # urlparse对URL进行拆分（举例说明下面这行代码的作用："http://www.baidu.com/index.php?username=guol"将会返回"http://www.baidu.com"）
    includeUrl = urlparse(includeUrl).scheme+"://"+urlparse(includeUrl).netloc
    
    internalLinks = []
    # 找出所有以"/"或includeUrl开头的链接，即内链
    for link in bsObj.findAll("a", href=re.compile("^(/|.*"+includeUrl+")")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                # 如果内链以"/"开头，则添加includeUrl
                if(link.attrs['href'].startswith("/")):
                    internalLinks.append(includeUrl+link.attrs['href'])
                # 如果内链以includeUrl开头，则不需要进行处理
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks
            
# 获取页面所有外链的列表
def getExternalLinks(bsObj, excludeUrl):
    externalLinks = []
    # 找出所有以"http"或"www"开头且不包含当前URL的链接
    for link in bsObj.findAll("a", href=re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

# 获取页面上某一个外链
def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bsObj = BeautifulSoup(html, "lxml")
    externalLinks = getExternalLinks(bsObj, urlparse(startingPage).netloc)
    
    # 如果当前页面没有外链，则随机进入页面上的一个内链，跳转到该页面后继续寻找外链
    if len(externalLinks) == 0:
        print("No external links, looking around the site for one")
        domain = urlparse(startingPage).scheme+"://"+urlparse(startingPage).netloc
        internalLinks = getInternalLinks(bsObj, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print("Random external link is: "+externalLink)
    followExternalOnly(externalLink)

followExternalOnly("http://oreilly.com")

Random external link is: https://www.youtube.com/user/OreillyMedia


URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

### 2.3.2 获取一个网站的所有内链和外链

![](https://raw.githubusercontent.com/blueliberty/Web-Scraping/master/Pictures/getAllExternalLinks.png)

In [28]:
allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = urlparse(siteUrl).scheme+"://"+urlparse(siteUrl).netloc
    bsObj = BeautifulSoup(html, "lxml")
    internalLinks = getInternalLinks(bsObj, domain)
    externalLinks = getExternalLinks(bsObj, domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            getAllExternalLinks(link)

allIntLinks.add("http://oreilly.com")
getAllExternalLinks("http://oreilly.com")

https://www.oreilly.com
http://www.oreilly.com/ideas
http://www.oreilly.com/learning
http://www.oreilly.com/conferences/
http://shop.oreilly.com/
http://members.oreilly.com
https://www.oreilly.com/topics
https://www.safaribooksonline.com/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170201+homepage+free+trial
https://www.safaribooksonline.com/enterprise/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170201+homepage+enterprise
https://www.safaribooksonline.com/accounts/login/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170203+homepage+sign+in
https://www.oreilly.com/people/129b9-ted-malaska?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170224+homepage+ted+malaska
https://www.safaribooksonline.com/search/?query=ted%20malaska&formats=live%20online%20training&highlight=true&extended_publisher_data=true&include_courses=true&field=authors&sort=date_added&utm_medium=content&utm_s

KeyboardInterrupt: 

In [31]:
len(allExtLinks)

130

In [32]:
len(allIntLinks)

13